## Przygotowanie sesji Spark
Zainicjowanie sesji Spark oraz stworzenie schematu bazy danych z której będziemy korzystać.

In [1]:
%env PYSPARK_SUBMIT_ARGS=--master local[*]  --jars /tmp/gcs-connector-hadoop2-1.9.17-shaded.jar,/tmp/google-cloud-nio-0.120.0-alpha-shaded.jar --conf spark.driver.port=29010 --conf spark.blockManager.port=29011  --conf spark.driver.host=jupyter-service- --conf spark.executorEnv.PYSPARK_PYTHON=python3 --packages org.biodatageeks:sequila_2.11:0.5.20,io.projectglow:glow-spark2_2.11:0.6.0,org.biodatageeks:seqtender_2.11:0.3.7 pyspark-shell


env: PYSPARK_SUBMIT_ARGS=--master local[*]  --jars /tmp/gcs-connector-hadoop2-1.9.17-shaded.jar,/tmp/google-cloud-nio-0.120.0-alpha-shaded.jar --conf spark.driver.port=29010 --conf spark.blockManager.port=29011  --conf spark.driver.host=jupyter-service- --conf spark.executorEnv.PYSPARK_PYTHON=python3 --packages org.biodatageeks:sequila_2.11:0.5.20,io.projectglow:glow-spark2_2.11:0.6.0,org.biodatageeks:seqtender_2.11:0.3.7 pyspark-shell


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master("local[2]") \
.config("spark.driver.host", "localhost") \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

Pobranie sesji Spark jest proste dla programisty korzystającego z notatnika, wymaga podania tylko kilku parametrów, ale faktyczna konfiguracja jest bardziej rozbudowana

In [3]:
import os
args = os.environ['PYSPARK_SUBMIT_ARGS'].replace("  ", "\n")
print(args)

--master local[*]
--jars /tmp/gcs-connector-hadoop2-1.9.17-shaded.jar,/tmp/google-cloud-nio-0.120.0-alpha-shaded.jar --conf spark.driver.port=29010 --conf spark.blockManager.port=29011
--conf spark.driver.host=jupyter-service- --conf spark.executorEnv.PYSPARK_PYTHON=python3 --packages org.biodatageeks:sequila_2.11:0.5.20,io.projectglow:glow-spark2_2.11:0.6.0,org.biodatageeks:seqtender_2.11:0.3.7 pyspark-shell


Mamy pobraną sesję sparkową. Powstały dodatkowe pody gotowe na realizację obliczeń. A jak zwolnić te zasoby?

In [ ]:
spark.stop()

Ponownie pobieramy sesję Spark. Będziemy z niej korzystać. Po zakończonej pracy należy pamiętać o zastopowaniu sesji.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

## Odczyt danych
Korzystając z sesji Spark można odczytać dane zapisane w lokalizacji dostępnej dla executorów (procesów obliczeniowych) koordynowanych przez Spark. 
Konieczne jest podanie ścieżki dostępowej do pliku i formatu danych (nie jest to jednoznazne z rozszerzeniem pliku).
Dystrybucja Spark udostępnia kilka tzw Data Sources, które odczytują i zapisują dane w określonych formatach (CSV, formaty kolumnowe: parquet/orc).
Data Sources zgodne z opracowanym interfejsem można samodzielnie tworzyć. Na dzisiejszych zajęciach będziemy korzystać z takich zdefiniowanych DS:
* FASTQDataSource
* BAMDataSource
* VCFDataSource

**UWAGA - powyższe sposoby odczytu plików nie są częścią głównej dystrybucji Spark. Wymagana jest dodatkowa konfiguracja.**

In [4]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-common-data2" # konstruujemy sciezke dostepowa do pliku
print(bucket)

gs://edugen-common-data2


In [5]:
!gsutil ls gs://edugen-common-data2

gs://edugen-common-data2/mother.fastq
gs://edugen-common-data2/mother.fq
gs://edugen-common-data2/anno/
gs://edugen-common-data2/bam/
gs://edugen-common-data2/fastq/
gs://edugen-common-data2/ref/
gs://edugen-common-data2/vcf/


In [15]:
! mkdir -p  data/fastq/

In [16]:
!gsutil cp gs://edugen-common-data2/fastq/* data/fastq/

Copying gs://edugen-common-data2/fastq/father.fastq...
Copying gs://edugen-common-data2/fastq/mother.fastq...                          
Copying gs://edugen-common-data2/fastq/son.fastq...                             
/ [3 files][ 79.0 MiB/ 79.0 MiB]    1.2 MiB/s                                   
Operation completed over 3 objects/79.0 MiB.                                     


---

`Co oznacza f przed cudzysłowem?`

`Czy jest różnica między stosowaniem apostrofu i cudzysłowu przy definicji zmiennych przechowywujących łańcuchy znaków?`

In [19]:
reads_path = f"data/fastq/*"  # * oznacza wszystkie pliki we wskazanej lokalizacji. Można podać konkretny plik
fastq_all = spark.read.load(reads_path, format="org.biodatageeks.sequila.datasources.FASTQ.FASTQDataSource")
fastq_all = fastq_all.select("sample_id","seq", "qual")

### Weryfikacja danych

In [20]:
type(fastq_all)      # jaki jest typ danych utworzonej zmiennej? 

pyspark.sql.dataframe.DataFrame

In [21]:
fastq_all.printSchema() # jaki jest schemat danych?

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)



In [22]:
len(fastq_all.columns)           # wymiary (liczba kolumn)

3

In [23]:
fastq_all.count()               # wymiary (liczba wierszy)

241114

In [24]:
fastq_all.explain(True)              #  plan wykonania

== Parsed Logical Plan ==
'Project [unresolvedalias('sample_id, None), unresolvedalias('seq, None), unresolvedalias('qual, None)]
+- Relation[sample_id#89,instrument_name#90,run_id#91,flowcell_id#92,lane#93,tile#94,pos_x#95,pos_y#96,filter_passed#97,control_num#98,index_seq#99,seq#100,qual#101] org.biodatageeks.sequila.datasources.FASTQ.SequencedFragmentRelation@16731808

== Analyzed Logical Plan ==
sample_id: string, seq: string, qual: string
Project [sample_id#89, seq#100, qual#101]
+- Relation[sample_id#89,instrument_name#90,run_id#91,flowcell_id#92,lane#93,tile#94,pos_x#95,pos_y#96,filter_passed#97,control_num#98,index_seq#99,seq#100,qual#101] org.biodatageeks.sequila.datasources.FASTQ.SequencedFragmentRelation@16731808

== Optimized Logical Plan ==
Project [sample_id#89, seq#100, qual#101]
+- Relation[sample_id#89,instrument_name#90,run_id#91,flowcell_id#92,lane#93,tile#94,pos_x#95,pos_y#96,filter_passed#97,control_num#98,index_seq#99,seq#100,qual#101] org.biodatageeks.sequila.dat

DataFrame jest abstrakcją nad innym typem danych (RDD), który jest podstawową rozproszoną strukturą danych. Poprzez DF możemy dostać się do rdd i zweryfikować na przykład liczbę partycji danych.

In [25]:
fastq_all.rdd.getNumPartitions() # liczba partycji (bloków danych)

4

### Podgląd danych

In [26]:
fastq_all.show()

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|
|   father|GTGAGAAACTGGCTCAT...|;::<=;<<<>;==><>=...|
|   father|TGGGAGTTAAATAGCAA...|=<<<?@>=>>>>>?>?>...|
|   father|TATTACATTTCAACGTG...|;;;;==>=<<=>=<7==...|
|   father|GCCTTTGCTAAATATGA...|==0>>=;>38;11=1<:...|
|   father|GTTACCAAATTTAGGGT...|;159,,3---1:<;;1-...|
|   father|AAGGACAGAAGCAATAA...|===70<>.>==>>=>>=...|
|   father|CAGGTGATCTCATAATT

In [27]:
fastq_all.show(5)  # pierwsze 5 wierszy

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [28]:
fastq_all.show(truncate=False) # bez skracania zawartości kolumn

+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|sample_id|seq                                                                                                                                                    |qual                                                                                                                                                   |
+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|father   |TTTGAGTGAGTTTATTAATCCTGAGTTCTAATTTGATTGCA

Widok "szerokich" tabel jest nieczytelny, w kolejnych częściach warsztatów zaradzimy temu korzystając z dodatkowej biblioteki.

## Dostęp do wybranych danych

#### Operacja projekcji (SELECT)

In [29]:
fastq_all.select("sample_id").show()

+---------+
|sample_id|
+---------+
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
+---------+
only showing top 20 rows



`Czy operacja select (sample_id) wpłynęła na oryginalny data frame fastq_all?`

In [30]:
fastq_all.printSchema()
fastq_all.show() 

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|
|   father|GTGAGAAACTGGCTCAT...|;::<=;<<<>;==><>=...|
|   father|TGGGAGTTAAATAGCAA...|=<<<?@>=>>>>>?>?>...|
|   father|TATTACATTTCAACGTG...|;;;;==>=<<=>=<7==...|
|   father|GCCTTTGCTAAATATGA...|==0>>=;>38;11=1<:...|
|   father|GTTACCA

Jeśli chcemy zachować wynik działania transformacji (w celu późniejszego wykorzystania) trzeba wynik zachować w zmiennej.

In [31]:
fastq_sample_only = fastq_all.select("sample_id")  

In [32]:
fastq_sample_only.printSchema()
fastq_sample_only.show()

root
 |-- sample_id: string (nullable = true)

+---------+
|sample_id|
+---------+
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
|   father|
+---------+
only showing top 20 rows



In [33]:
fastq_all.select("sample_id","seq").show()

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|
|   father|ACTGTCAATATTAGACA...|
|   father|CCTCCCATTTTACTTTT...|
|   father|ATTAATTAGATTTCAGT...|
|   father|GTAGGCAAAGCCTCACA...|
|   father|TCTATTCAAAATATTTG...|
|   father|ATGTGGTTTGCAAATAT...|
|   father|ATGCGGAACTGATGAGT...|
|   father|CTATGAAACAATAGTCT...|
|   father|GTGAGAAACTGGCTCAT...|
|   father|TGGGAGTTAAATAGCAA...|
|   father|TATTACATTTCAACGTG...|
|   father|GCCTTTGCTAAATATGA...|
|   father|GTTACCAAATTTAGGGT...|
|   father|AAGGACAGAAGCAATAA...|
|   father|CAGGTGATCTCATAATT...|
|   father|TGAAGTTGGGGTGCACC...|
|   father|ACATATTACGCGGAATC...|
|   father|TATTACTATTTCTGTAT...|
|   father|AGTTACTTCTCTGGCCA...|
+---------+--------------------+
only showing top 20 rows



Powiedzmy, że interesują nas wszystkie kolumny poza qual. Jak to zrobic? Można wylistować wszystkie kolumny poza qual, ale to uciazliwe. Mozna skorzystac z operacji drop.

In [34]:
fastq_no_qual=fastq_all.drop('qual')

`Czy operacja usunięcia kolumny qual wpłynęła na oryginalny data frame fastq_all?`

In [35]:
fastq_no_qual.printSchema()
fastq_no_qual.show()

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|
|   father|ACTGTCAATATTAGACA...|
|   father|CCTCCCATTTTACTTTT...|
|   father|ATTAATTAGATTTCAGT...|
|   father|GTAGGCAAAGCCTCACA...|
|   father|TCTATTCAAAATATTTG...|
|   father|ATGTGGTTTGCAAATAT...|
|   father|ATGCGGAACTGATGAGT...|
|   father|CTATGAAACAATAGTCT...|
|   father|GTGAGAAACTGGCTCAT...|
|   father|TGGGAGTTAAATAGCAA...|
|   father|TATTACATTTCAACGTG...|
|   father|GCCTTTGCTAAATATGA...|
|   father|GTTACCAAATTTAGGGT...|
|   father|AAGGACAGAAGCAATAA...|
|   father|CAGGTGATCTCATAATT...|
|   father|TGAAGTTGGGGTGCACC...|
|   father|ACATATTACGCGGAATC...|
|   father|TATTACTATTTCTGTAT...|
|   father|AGTTACTTCTCTGGCCA...|
+---------+--------------------+
only showing top 20 rows



## Wartości unikalne
Jeśli chcemy uzyskać unikalne wartości w określonych kolumnach korzystamy z metody distinct().
Operacje na DF można łańcuchowo łączyć, zatem na wyniku działania select() można wywołać kolejne metody.

In [36]:
fastq_all.select('sample_id').distinct().show()

+---------+
|sample_id|
+---------+
|   father|
|   mother|
|      son|
+---------+



In [37]:
fastq_all.count()

241114

# Sortowanie

Do sortowania służy metoda orderBy. Domyślne sortowanie jest rosnące.

In [38]:
fastq_all.select('sample_id').distinct().orderBy('sample_id').show()

+---------+
|sample_id|
+---------+
|   father|
|   mother|
|      son|
+---------+



In [39]:
fastq_all.select('sample_id').distinct().orderBy('sample_id', ascending=False).show()

+---------+
|sample_id|
+---------+
|      son|
|   mother|
|   father|
+---------+



In [40]:
fastq_all.orderBy('sample_id', 'seq', ascending=False).show()  # kierunek sortowania jest wspólny dla listy kolumn

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|TTTTTTTTTTTTTTTTT...|..--61<:::5:+5:<1...|
|      son|TTTTTTTTTTTTTTTTT...|;:;:=++<<=++=<<=+...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=<<+1:1<<9<2+...|
|      son|TTTTTTTTTTTTTTTTT...|;;--=+<1<2+<:<<=+...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<==<1+:<<<<:<<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=+<:<=:511:69...|
|      son|TTTTTTTTTTTTTTTTT...|.:-<61<<56+<<1++<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=9+5=,<+<+6<<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=<<1+=+1:5+<1...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<==<<<<<<+...|
|      son|TTTTTTTTTTTTTTTTT...|9:-:==:<5:15:<:+9...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<56<<1<<<<...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<<2<<:<<<<...|
|      son|TTTTTTTTTTTTTTTTG...|;:;<=9++5=1+++<<1...|
|      son|TTTTTTTTTTTTTTTTG...|;:8<=+<5+255<1<<8...|
|      son|TTTTTTTTTTTTTTTTC

In [41]:
fastq_all.orderBy('sample_id', ascending=False).orderBy('seq', ascending=True).show() # sortowanie malejace i rosnące na dwóch roznych kolumnach 

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   mother|AAAAAAAAAAAAAAAAA...|====??=>>>8>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;<9<=>>==:=======...|
|   father|AAAAAAAAAAAAAAAAA...|/;.:/78844<8,=<47...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>>>>>>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|==;=??=>=>9>:>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>>>>>>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>===9<==7===...|
|   mother|AAAAAAAAAAAAAAAAA...|==;<=?>>=>9>9>==>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>======<...|
|   mother|AAAAAAAAAAAAAAAAA...|===<??>>>>>>>>>:>...|
|   mother|AAAAAAAAAAAAAAAAA...|===<??>>>>>>=>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>=======...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>=>><>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;9:/9;:=.83=####...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>=======...|
|   mother|AAAAAAAAAAAAAAAAA

### Filtrowanie wynikow
Nasz zbiór danych posiada odczyty z 3 próbek. Ograniczmy się do wybranych próbek.

In [42]:
fastq_mother = fastq_all.filter("sample_id = 'mother'")

`Czy już odbył się odczyt danych z fastq?`

In [43]:
fastq_mother.show()

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   mother|TGGAGTTCTAAATTGTG...|==:=?>=>?>>>>>>>>...|
|   mother|GCATGTGCAAGTATCTT...|;;<<9>>>>9>:=<=?<...|
|   mother|GCCATTAGACGACATTT...|=<=2?<0?=;5>>?>70...|
|   mother|AGGCAGGACTAGTTCCT...|::;,::*-=0:>88-+;...|
|   mother|ATAAGGGGGCCACTTTT...|==8=??>>>?>?>@>==...|
|   mother|TGAAAAACCACATGGAA...|/99;;9=8=>=-2=::3...|
|   mother|GAAATGCTTAATTACCC...|====2??@<=>>==8=0...|
|   mother|CCACAAAAGCCTTTATC...|;;<6<.>=>2+>=8,=9...|
|   mother|CAACTTAACAAATTCAT...|=>;<?>?>>?>=>=>?>...|
|   mother|TTCTCCTAAGTGAAATA...|;:;=9>?==?===<==>...|
|   mother|AGAATTACTGTTTGACT...|</;=:>>>?9/:7;<>4...|
|   mother|AGATATAAATTATATTG...|:.9/;-?.=8+>->-+7...|
|   mother|TTTGAACCTTTACTACA...|====??>>?==?>?=>?...|
|   mother|TCAAGGACTAAGATATA...|;0:<=>>=>?<=9=>=>...|
|   mother|AGTAAAGGTTCTAGTTA...|<=;==?>>=><@7<;9=...|
|   mother|GAGGGTGTCATGGGAAC

In [44]:
fastq_mother.select('sample_id').distinct().show()

+---------+
|sample_id|
+---------+
|   mother|
+---------+



In [45]:
fastq_mother.count()

133160

Warunki można łączyć spójnikami logicznymi. 
Można używać
* operatorów arytmetycznych (=, !=, >, >=, <, <=)
* przynależności do zbioru (IN/NOT IN) 
* porównania znaków (LIKE/NOT LIKE)
* przyrównania do wartości NULL (IS NULL/ IS NOT NULL)

Przy korzystaniu z LIKE można użyć % jako oznaczenie dowolnego ciągu znaków.

Konstrukcja warunku w metodzie filter() jak taka jak w klauzuli WHERE W SQL.

Pokaż odczyty spełniające warunek ze nazwa instrumentu jest pusta, run_id jest >=0 a odczyt zaczyna sie od liter GCA. Pokaz tylko kolumny z filtra oraz nazwe probki

In [46]:
fastq_mother.filter('seq LIKE "GCA%"').select('sample_id',  'seq').show()

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   mother|GCATGTGCAAGTATCTT...|
|   mother|GCAGGTACTCATGTTCA...|
|   mother|GCACATCCTGGCCTCCT...|
|   mother|GCAACCTCCACCTCCTA...|
|   mother|GCAGAAGGTGCAAAAGC...|
|   mother|GCATCGAATTTTTTTTC...|
|   mother|GCACTTCCCTGGCCACA...|
|   mother|GCAGTCACCTTCCCAGC...|
|   mother|GCACCTCTGATGCTGGA...|
|   mother|GCAGAAAGCTGGGTGCA...|
|   mother|GCATAATAGACACTAGG...|
|   mother|GCAATCAATCAGATGGC...|
|   mother|GCAACTTCAGTAAAGTC...|
|   mother|GCAAACGAACACAGGAA...|
|   mother|GCATATCATTTCCCATA...|
|   mother|GCATTTTCACTTGCATG...|
|   mother|GCAGTTACAGTCTTAGA...|
|   mother|GCAACTTATAAAATGGG...|
|   mother|GCAACATATAAAATGGG...|
|   mother|GCAACGGACAAGGAGAA...|
+---------+--------------------+
only showing top 20 rows



In [47]:
fastq_mother.select('sample_id', 'seq').filter('seq LIKE "GCA%"').show() # kolejnosc select i filter bez znaczenia

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   mother|GCATGTGCAAGTATCTT...|
|   mother|GCAGGTACTCATGTTCA...|
|   mother|GCACATCCTGGCCTCCT...|
|   mother|GCAACCTCCACCTCCTA...|
|   mother|GCAGAAGGTGCAAAAGC...|
|   mother|GCATCGAATTTTTTTTC...|
|   mother|GCACTTCCCTGGCCACA...|
|   mother|GCAGTCACCTTCCCAGC...|
|   mother|GCACCTCTGATGCTGGA...|
|   mother|GCAGAAAGCTGGGTGCA...|
|   mother|GCATAATAGACACTAGG...|
|   mother|GCAATCAATCAGATGGC...|
|   mother|GCAACTTCAGTAAAGTC...|
|   mother|GCAAACGAACACAGGAA...|
|   mother|GCATATCATTTCCCATA...|
|   mother|GCATTTTCACTTGCATG...|
|   mother|GCAGTTACAGTCTTAGA...|
|   mother|GCAACTTATAAAATGGG...|
|   mother|GCAACATATAAAATGGG...|
|   mother|GCAACGGACAAGGAGAA...|
+---------+--------------------+
only showing top 20 rows



<div class="alert alert-block alert-warning">

<b>Zadanie 2_2:</b>Napisz polecenie które policzy ile jest rekordów dla próbki syna które spełniają warunki, że sekwencja odczytu konczy się na TGG a qual zaczyna się od ==. </div>



## Używanie funkcji, kolumny wyliczane

Dostępne są funkcje skalarne (przykład: ROUND, UPPER, CURRENT_DATE) oraz agregujące (MIN, MAX, AVG, SUM, COUNT).
Niektóre funkcje są dostępne "od razu" bez dodatkowych poleceń import. 
Lista funkcji znajduje się : https://spark.apache.org/docs/latest/api/sql/index.html 

In [48]:
fastq_all.selectExpr("*").show()  # pokaż wszystkie kolumny tego DF

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|
|   father|GTGAGAAACTGGCTCAT...|;::<=;<<<>;==><>=...|
|   father|TGGGAGTTAAATAGCAA...|=<<<?@>=>>>>>?>?>...|
|   father|TATTACATTTCAACGTG...|;;;;==>=<<=>=<7==...|
|   father|GCCTTTGCTAAATATGA...|==0>>=;>38;11=1<:...|
|   father|GTTACCAAATTTAGGGT...|;159,,3---1:<;;1-...|
|   father|AAGGACAGAAGCAATAA...|===70<>.>==>>=>>=...|
|   father|CAGGTGATCTCATAATT

Dodanie dwóch dodatkowych kolumn wyliczanych 

In [49]:
fastq_all.selectExpr("*", "length(seq) as len_seq", "length(qual) as len_qual" ).show() ## dodanie dwóch kolumn wyliczanych przy uzyciu funkcji LENGTH

+---------+--------------------+--------------------+-------+--------+
|sample_id|                 seq|                qual|len_seq|len_qual|
+---------+--------------------+--------------------+-------+--------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|    151|     151|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|    151|     151|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|    151|     151|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|    151|     151|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|    151|     151|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|    151|     151|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|    151|     151|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|    151|     151|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|    151|     151|
|   father|GTGAGAAACTGGCTCAT...|;::<=;<<<>;==><>=...|    151|     151|
|   father|TGGGAGTTAAATAGCAA...|=<<<?@>=>>>>>?>?>...|    151|     151|
|   fa

Alias - nadanie kolumnie lub kolumnie wyliczanej nazwy

In [50]:
extended_fastq = fastq_all.selectExpr("*", "length(seq) as len_s", "length(qual) as len_q" ) # AS alias

In [51]:
extended_fastq.printSchema()

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)
 |-- len_s: integer (nullable = true)
 |-- len_q: integer (nullable = true)



Dodanie nowej kolumny, dla każdego wiersza zostanie dodana wartość zwracana przez funkcję current_date()

In [52]:
from pyspark.sql.functions import current_date
extended_fastq.withColumn ("date", current_date()).show()

+---------+--------------------+--------------------+-----+-----+----------+
|sample_id|                 seq|                qual|len_s|len_q|      date|
+---------+--------------------+--------------------+-----+-----+----------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|  151|  151|2023-05-26|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|  151|  151|2023-05-26|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|  151|  151|2023-05-26|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|  151|  151|2023-05-26|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|  151|  151|2023-05-26|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|  151|  151|2023-05-26|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|  151|  151|2023-05-26|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|  151|  151|2023-05-26|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|  151|  151|2023-05-26|
|   father|GTGAGAAACTGGCTCAT...|;::<=;<<<>;==><>=...|  151|  151|2023-05-26|

Dodanie kolumny o stałej wartości dla każdej wartości wymaga wykorzystania funkcji lit (), która przekształci stała wartość w kolumnę.

In [53]:
from pyspark.sql.functions import lit

extended_fastq.withColumn("imported_by", lit(user_name)).withColumn("format", lit('FASTQ')).show()

+---------+--------------------+--------------------+-----+-----+-----------+------+
|sample_id|                 seq|                qual|len_s|len_q|imported_by|format|
+---------+--------------------+--------------------+-----+-----+-----------+------+
|   father|TTTGAGTGAGTTTATTA...|=<<:=@=>>=<=<>>=>...|  151|  151|     jovyan| FASTQ|
|   father|ACTGTCAATATTAGACA...|;9<;<>9<=<=<=><<>...|  151|  151|     jovyan| FASTQ|
|   father|CCTCCCATTTTACTTTT...|==>=???>===>=?===...|  151|  151|     jovyan| FASTQ|
|   father|ATTAATTAGATTTCAGT...|:;:<=><=><=;<;>>;...|  151|  151|     jovyan| FASTQ|
|   father|GTAGGCAAAGCCTCACA...|=<==>>?>>?=>?>?=>...|  151|  151|     jovyan| FASTQ|
|   father|TCTATTCAAAATATTTG...|/9<9=:,<44,,::;<;...|  151|  151|     jovyan| FASTQ|
|   father|ATGTGGTTTGCAAATAT...|==:<>?>==><?>=>>>...|  151|  151|     jovyan| FASTQ|
|   father|ATGCGGAACTGATGAGT...|:;7<65-=9>9,;4;2;...|  151|  151|     jovyan| FASTQ|
|   father|CTATGAAACAATAGTCT...|=>==>>=>=:>>:=>=?...|  151|  151|

<div class="alert alert-block alert-warning">

<b>Zadanie 2_3:</b>

Napisz polecenie które stworzy ramkę danych zawierającą sklejenie wartości dwóch kolumn (sample_id) oraz daty eksperymentu (dodaj kolumne z wartościami 2019-01-15) . W wynikach chcemy mieć tylko dane matki i ojca. Kolumny wynikowe: nazwa próbki, seq, qual, data eksperymentu oraz scalona nazwa probki oraz data eksperymenty (np father-2019-01-15). Posortuj po nazwie próbki. Pokaż schemat ramki. Upewnij się, że data eksperymentu jest typu date. 

* Zwróć uwagę na potrzebę konwersji ciągu znaków na datę
</div>


### Instrukcje warunkowe przy kolumnach wyliczanych

In [54]:
fastq_dates=fastq_all.selectExpr('*', 'if(sample_id = "son",to_date("2018-11-10"), to_date("2019-01-15")) as experiment_date')

<div class="alert alert-block alert-warning">
<b>Zadanie 2_4:</b> Napisz polecenia, które zweryfikuje czy daty eksperymentów zostały dodane poprawnie </div>


## Grupowanie

In [55]:
fastq_all.groupBy("sample_id").show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [56]:
type(fastq_all.groupBy("sample_id")) # to nie jest DF

pyspark.sql.group.GroupedData

In [57]:
sample_count=fastq_all.groupBy("sample_id").count()

In [58]:
type(sample_count)

pyspark.sql.dataframe.DataFrame

In [59]:
sample_count.show()

+---------+------+
|sample_id| count|
+---------+------+
|   father| 53770|
|   mother|133160|
|      son| 54184|
+---------+------+



In [60]:
sample_count.orderBy("count").show()

+---------+------+
|sample_id| count|
+---------+------+
|   father| 53770|
|      son| 54184|
|   mother|133160|
+---------+------+



<div class="alert alert-block alert-warning">
<b>Zadanie 2_5:</b> Napisz funkcje, ktora znajdzie rozklady jakosci dla 1, 2, 3 i 4 pozycji odczytu.
*Nastepnie zaprezentuj wyniki w postaci serii histogramów </div>

### Mapowanie do genomu referencyjnego

UWAGA - na dzisiejszych zajęciach ten kod nie będzie uruchamiany

Wykonamy mapowanie do genomu referencyjnych korzystając z rozproszenia danych miedzy procesy obliczeniowe sparka.

Przygotowanie ścieżek do plików.

In [ ]:
#import os
#user_name = os.environ.get('USER')
#bucket = f"gs://edugen-lab-{user_name}"

#reads_file_path = f"{bucket}/fastq/mother.fastq"
#ref_path = "/mnt/data/mapping/ref/ref.fasta"

Konstruujemy komendę, która będzie uruchamiana na procesach obliczeniowych. Potrzebne narzędzia muszą być dostępne na węzłach obliczeniowych.

In [ ]:
#command = f'bwa mem -p {ref_path} - | samtools fixmate -m - - | samtools sort  | samtools markdup -r -S - -  | samtools addreplacerg  -r "ID:S1" -r "SM:S1"  -r "PL:ILLUMINA" - | samtools view -b -'

Żeby wykonać rozproszone obliczenia na danych genomicznych nalezy wykorzystac dodatkową bibliotekę.

In [ ]:
#from pyseqtender import SeqTenderAlignment

#seq_aligner = SeqTenderAlignment(spark, reads_file_path, command)
#alignments_rdd = seq_aligner.pipe_reads()

Zapisujemy plik na kubełek.

In [ ]:
#bam_file_path = f"{bucket}/bam/mother10.bam"
#seq_aligner.save_reads(bam_file_path, alignments_rdd)

In [ ]:
#!gsutil ls gs://edugen-lab-$USER/bam

<div class="alert alert-block alert-warning">
<b>Zadanie 2_6:</b> Na podstawie notatników z zajęć z genomiki wyświetl fragment pliku BAM w widge'cie IGV.  </div>



Kończymy notatniki, należy zamknąć sesję.

In [ ]:
spark.stop()